# Example LC fit using the FAST DB API

This example assumes you've done the steps in the `using_fastdb_client.ipynb` example first. If not, go there and do those first. 


## Setting up your environment

`fastdb_client.py` requires libraries that are not yet in the standard desc time domain environment.  However, they are in the current dev environment.  On Perlmutter, you can get yourself in this environment with
```
source $CFS/lsst/groups/TD/setup_td_dev.sh
```

On your local machine, install `pycryptodome` and modify the `sys.path.insert` to point to the part for your local `fastdb_client.py` file.

In [ ]:
import sys

# sys.path.insert( 0, '/global/cfs/cdirs/lsst/groups/TD/SOFTWARE/fastdb_deployment/fastdb_client' )
sys.path.insert( 0, '/Users/gnarayan/work/FASTDB/client/' )

#...and import!
from fastdb_client import FASTDBClient

# Also import other stuff we'll need
import io
import time
import pandas
import json


In [ ]:
# Create your connection to fastdb.  This will log you in.
# the connection info is in the .fastdb.ini file 
fdb = FASTDBClient( "rknop-dev2" )

## WEB API Endpoints

You use the fastdb client to connect to web API endpoints on the server.  All of these will eventually be documented, but for now, here are some examples of endpoints that you can hit.

In [ ]:
# /getprocvers lists all the known processing versions and processing version aliases. 
# (There are at most as many different processing versions as the list you get back,
# because one processing version may have multiple aliases.)
res = fdb.post( "/getprocvers" )
print( res )

In [ ]:
# You can get the lightcurve for an object if you know it's object ID with the /ltcv/getltcv 
# endpoint.  Give it the processing version and the diaobjectid as additional slash-separated
# url components.
t0 = time.perf_counter()
ltcvdict = fdb.post( "/ltcv/getltcv/dp1/609782208097419314" )
print( f"/ltcv/getltcv took {time.perf_counter()-t0:.2f} seconds" )

# The dictionary you get back has keys 'status', 'objectinfo' and 'ltcv':
print( f"Got back ltcvdict with keys {ltcvdict.keys()}" )

print( f"Object info: {json.dumps( ltcvdict['objinfo'], indent=4 )}" )

ltcv = pandas.DataFrame( ltcvdict['ltcv'] )
ltcv

In [ ]:
# You can search for lightcurves with /objectsearch/<procver>
# This one requires you to pass some search data as a dict:

res = fdb.post( "/objectsearch/dp1", json={"ra": 52.83825, "dec": -28.27990, "radius": 1.} )
objs = pandas.DataFrame(res)
objs